# Tidy Data
Dataset from [Kaggle.com](https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war?select=russia_losses_equipment.csv)

In [8]:
import pandas as pd

In [39]:
df = pd.read_csv("./data/russia_losses_equipment.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   33 non-null     object 
 1   day                    33 non-null     int64  
 2   aircraft               33 non-null     int64  
 3   helicopter             33 non-null     int64  
 4   tank                   33 non-null     int64  
 5   APC                    33 non-null     int64  
 6   field artillery        33 non-null     int64  
 7   MRL                    33 non-null     int64  
 8   military auto          33 non-null     int64  
 9   fuel tank              33 non-null     int64  
 10  drone                  33 non-null     int64  
 11  naval ship             33 non-null     int64  
 12  anti-aircraft warfare  33 non-null     int64  
 13  special equipment      14 non-null     float64
 14  mobile SRBM system     4 non-null      float64
dtypes: float

In [40]:
df.head()

,date,day,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment,mobile SRBM system
0,2022-02-25,2,10,7,80,516,49,4,100,60,0,2,0,NaN,NaN
1,2022-02-26,3,27,26,146,706,49,4,130,60,2,2,0,NaN,NaN
2,2022-02-27,4,27,26,150,706,50,4,130,60,2,2,0,NaN,NaN
3,2022-02-28,5,29,29,150,816,74,21,291,60,3,2,5,NaN,NaN
4,2022-03-01,6,29,29,198,846,77,24,305,60,3,2,7,NaN,NaN


This dataset isn't tidy. There are variables in the columns such as `aircraft` and `tank`. These should go in a column like `equipment_type`. This is actually a good example of one of the common problems Wickham talked about, that is, *"column headers are values, not variable names."*

In [41]:
df = pd.melt(df, id_vars=list(df.columns)[:2], value_vars=list(df.columns)[2:], var_name='equipment_type', value_name='count')

This operation could use some explanation. `id_vars=list(df.columns)[:2]` identifies columns 0 up to but not including 2 as ids for the columns I'm going to melt and turn into two seperate sets of values. `value_vars=list(df.columns)[2:]` identifies every column from index 2 onward as the columns to melt and turn into values. `var_name='equipment_type'` sets the column names (which should be variables/values) as data in a new column `equipment_type`. `value_name='count'` sets the data that was in those columns as the data of the new column `count`.

In [42]:
# Have to use 'Int64' workaround since pandas integer columns normally cannot handle missing values.
# https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int
df['count'] = df['count'].astype('Int64')
df

,date,day,equipment_type,count
0,2022-02-25,2,aircraft,10
1,2022-02-26,3,aircraft,27
2,2022-02-27,4,aircraft,27
3,2022-02-28,5,aircraft,29
4,2022-03-01,6,aircraft,29
...,...,...,...,...
424,2022-03-25,30,mobile SRBM system,<NA>
425,2022-03-26,31,mobile SRBM system,2
426,2022-03-27,32,mobile SRBM system,4
427,2022-03-28,33,mobile SRBM system,4


Here, I `melt` the dataframe so that the variables for equipment in put under `equipment_type` and the associated count for each observation is placed in the `count` column. Then, because it doesn't make sense to count half of some equipment to be lost, I changed the data type of the `count` column to be an integer.

In [43]:
df = df.set_index(['date', 'equipment_type'])
df.head()

,,day,count
date,equipment_type,,
2022-02-25,aircraft,2,10
2022-02-26,aircraft,3,27
2022-02-27,aircraft,4,27
2022-02-28,aircraft,5,29
2022-03-01,aircraft,6,29


The default index column created by pandas is not needed since a multi-index of `date` and `equipment_type` can serve that role. The default index served no purpose.

In [45]:
df

,,day,count
date,equipment_type,,
2022-02-25,aircraft,2,10
2022-02-26,aircraft,3,27
2022-02-27,aircraft,4,27
2022-02-28,aircraft,5,29
2022-03-01,aircraft,6,29
...,...,...,...
2022-03-25,mobile SRBM system,30,<NA>
2022-03-26,mobile SRBM system,31,2
2022-03-27,mobile SRBM system,32,4


As you can see, the tidied dataset is easy to understand and maintains each observation that was in the original messy dataset.

The transformed dataset is *tidy* by Wickham's definition because:
1. Each variable, in this case the **type** and **count** of equipment lost, forms columns associated with those variables.
2. Each observation forms a row. Even if the observation has missing data, the observation was kept to maintain the fact that an observation occurred. 
3. Since the dataset only has one type of observational unit, equipment loss over time, it is kept to only one table.